In [21]:
import gzip
import json
import requests
from io import StringIO

# try:
#     from cStringIO import StringIO
# except:
#     from StringIO import StringIO

# Let's fetch the Common Crawl FAQ using the CC index
resp = requests.get('http://index.commoncrawl.org/CC-MAIN-2015-27-index?url=http%3A%2F%2Fcommoncrawl.org%2Ffaqs%2F&output=json')
resp_ = resp.content.strip().split(bytes("\n", encoding="utf-8"))

pages = [json.loads(x) for x in resp_]
# # Multiple pages may have been found - we're only interested in one
page = pages[0]

In [23]:
page

{'urlkey': 'org,commoncrawl)/faqs',
 'timestamp': '20150701063137',
 'url': 'http://commoncrawl.org/faqs/',
 'mime': 'text/html',
 'status': '200',
 'digest': 'ANEUZJSAIX4UDD2PRWXB2SEIKXBVXKO2',
 'length': '4406',
 'offset': '53124164',
 'filename': 'crawl-data/CC-MAIN-2015-27/segments/1435375094690.4/warc/CC-MAIN-20150627031814-00274-ip-10-179-60-89.ec2.internal.warc.gz'}

In [28]:

# If we print this, we'll see the JSON representation of the response
# Most important is the file path to read and the location within the large file that the GZIP response exists
print('JSON response from index.commoncrawl.org')
print('---')
print(page)

# We need to calculate the start and the end of the relevant byte range
# (each WARC file is composed of many small GZIP files stuck together)
offset, length = int(page['offset']), int(page['length'])
offset_end = offset + length - 1
# We'll get the file via HTTPS so we don't need to worry about S3 credentials
# Getting the file on S3 is equivalent however - you can request a Range
prefix = 'https://aws-publicdatasets.s3.amazonaws.com/'
# We can then use the Range header to ask for just this set of bytes
resp = requests.get(prefix + page['filename'], headers={'Range': 'bytes={}-{}'.format(offset, offset_end)})

# The page is stored compressed (gzip) to save space
# We can extract it using the GZIP library
#raw_data = json.loads(resp.content.decode("utf-8"))


JSON response from index.commoncrawl.org
---
{'urlkey': 'org,commoncrawl)/faqs', 'timestamp': '20150701063137', 'url': 'http://commoncrawl.org/faqs/', 'mime': 'text/html', 'status': '200', 'digest': 'ANEUZJSAIX4UDD2PRWXB2SEIKXBVXKO2', 'length': '4406', 'offset': '53124164', 'filename': 'crawl-data/CC-MAIN-2015-27/segments/1435375094690.4/warc/CC-MAIN-20150627031814-00274-ip-10-179-60-89.ec2.internal.warc.gz'}


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [34]:
raw_data = json.load(resp.content)
f = gzip.GzipFile(fileobj=raw_data)

# What we have now is just the WARC response, formatted:
data = f.read()
warc, header, response = data.strip().split('\r\n\r\n', 2)
#
# print 'WARC headers'
# print '---'
# print warc[:100]
# print '---'
# print 'HTTP headers'
# print '---'
# print header[:100]
# print '---'
# print 'HTTP response'
# print '---'
# print response[:100]

AttributeError: 'bytes' object has no attribute 'read'

In [44]:
import requests
import argparse
import time
import json
#import StringIO
from io import StringIO
import gzip
import csv
import codecs
from bs4 import BeautifulSoup
import sys
#sys.setdefaultencoding('utf8')

# parse the command line arguments
#ap = argparse.ArgumentParser()
#ap.add_argument("-d","--domain",required=True,help="The domain to target ie. cnn.com")
#args = vars(ap.parse_args())

# https://commoncrawl.github.io/cc-crawl-statistics/plots/domains
#domain = args['domain']
domain = "theguardian.com"

# list of available indices
index_list = ["2014-52","2015-06","2015-11","2015-14","2015-18","2015-22","2015-27"]

def search_domain(domain):

    record_list = []

    print("[*] Trying target domain: %s" % domain)

    for index in index_list:

        print("[*] Trying index %s" % index)

        cc_url  = "http://index.commoncrawl.org/CC-MAIN-%s-index?" % index
        cc_url += "url=%s&matchType=domain&output=json" % domain

        response = requests.get(cc_url)

        if response.status_code == 200:

            records = response.content.splitlines()

            for record in records:
                record_list.append(json.loads(record))

            print("[*] Added %d results." % len(records))


    print("[*] Found a total of %d hits." % len(record_list))

    return record_list

results = search_domain(domain)

[*] Trying target domain: theguardian.com
[*] Trying index 2014-52
[*] Added 14287 results.
[*] Trying index 2015-06
[*] Added 12169 results.
[*] Trying index 2015-11
[*] Trying index 2015-14
[*] Added 14327 results.
[*] Trying index 2015-18
[*] Added 12263 results.
[*] Trying index 2015-22
[*] Added 14573 results.
[*] Trying index 2015-27
[*] Added 12686 results.
[*] Found a total of 80305 hits.


In [131]:
results[0]["url"]

'http://www.theguardian.com/accenture-partner-zone/analytics-in-action-breakthroughs-barriers-roi'

In [133]:
from tqdm.notebook import tqdm

def download_page(record):

    offset, length = int(record['offset']), int(record['length'])
    offset_end = offset + length - 1

    # We'll get the file via HTTPS so we don't need to worry about S3 credentials
    # Getting the file on S3 is equivalent however - you can request a Range
    prefix = 'https://aws-publicdatasets.s3.amazonaws.com/'

    # We can then use the Range header to ask for just this set of bytes
    resp = requests.get(prefix + record['filename'], headers={'Range': 'bytes={}-{}'.format(offset, offset_end)})

    import io
    raw_data = io.StringIO(resp.content.decode("utf-8"))
    f = gzip.GzipFile(fileobj=(raw_data))


    # What we have now is just the WARC response, formatted:

    data = f.read()

    response = ""
    response.decode("utf-8")

    if len(data):
        try:
            warc, header, response = data.strip().split('\r\n\r\n', 2)
        except:
            pass

    return response



sample = results[0:10]
with tqdm(total=(len(sample)), position=0, leave=True) as pbar:
    for record in sample:
        html_content = download_page(record)
        print("[*] Retrieved %d bytes for %s" % (len(html_content),record['url']))

        #link_list = extract_external_links(html_content,link_list)

        pbar.update()

  0%|          | 0/10 [00:00<?, ?it/s]

TypeError: can't concat str to bytes

In [112]:
html_content

''

In [94]:
# Downloads a page from Common Crawl - adapted graciously from @Smerity - thanks man!
# https://gist.github.com/Smerity/56bc6f21a8adec920ebf

def extract_external_links(html_content,link_list):

    parser = BeautifulSoup(html_content)

    links = parser.find_all("a")

    if links:

        for link in links:
            href = link.attrs.get("href")

            if href is not None:

                if domain not in href:
                    if href not in link_list and href.startswith("http"):
                        print("[*] Discovered external link: %s" % href)
                        link_list.append(href)

    return link_list

#record_list = search_domain(domain)
link_list   = []

for record in results:

    html_content = download_page(record)

    print("[*] Retrieved %d bytes for %s" % (len(html_content),record['url']))

    link_list = extract_external_links(html_content,link_list)


print("[*] Total external links discovered: %d" % len(link_list))

with codecs.open("%s-links.csv" % domain,"wb",encoding="utf-8") as output:

    fields = ["URL"]

    logger = csv.DictWriter(output,fieldnames=fields)
    logger.writeheader()

    for link in link_list:
        logger.writerow({"URL":link})

AttributeError: 'str' object has no attribute 'read'